In [1]:
# Importaciones:
# Beautifulsoup
from bs4 import BeautifulSoup

# Requests
import requests

import pandas as pd
import numpy as np
import tqdm as tqdm
from time import sleep

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar 

In [2]:
ciudades_espana = [
    "Madrid",
]

# ciudades_espana = [
#     "Madrid",
#     "Barcelona",
#     "Valencia",
#     "Sevilla",
#     "Zaragoza",
#     "Málaga",
#     "Murcia",
#     "Palma de Mallorca",
#     "Las Palmas de Gran Canaria",
#     "Bilbao",
#     "Alicante",
#     "Córdoba",
#     "Valladolid",
#     "Vigo",
#     "Gijón",
#     "L'Hospitalet de Llobregat",
#     "La Coruña",
#     "Granada",
#     "Vitoria-Gasteiz",
#     "Elche",
# ]



In [29]:
driver=webdriver.Chrome()
url_wunder = "https://www.wunderground.com/history"
    # Abrimos navegador y maximizamos página en la url
driver.get(url_wunder)
driver.maximize_window()
    # Aceptacion de cookies
sleep(2)
iframe=WebDriverWait(driver,10).until(EC.presence_of_element_located(('xpath','//*[@id="sp_message_iframe_1165301"]')))
driver.switch_to.frame(iframe)
    #seleccionar busqueda
try:
    driver.find_element("css selector","#notice > div.message-component.message-row.cta-buttons-container > div.message-component.message-column.cta-button-column.reject-column > button").click()
except:    
    print('No encuentro el boton')
sleep(2)
driver.switch_to.default_content()
for ciudad in ciudades_espana:
    try:
        driver.find_element("css selector","#historySearch").send_keys({ciudad}, Keys.TAB)
    except:    
        print('No encuentro el boton')
    sleep(2)
    try:
        driver.find_element("css selector","#dateSubmit").click()
    except:    
        print('No encuentro el boton View')
    sleep(2)
    try:
        driver.find_element("css selector","#dateSubmit").click()
    except:    
        print('No encuentro el boton View')
    sleep(5)
    driver.find_element('xpath','//*[@id="inner-content"]/div[2]/div[1]/div[1]/div[1]/div/lib-link-selector/div/div/div/a[3]').click()
    try:
        driver.find_element("css selector","#inner-content > div.region-content-main > div.row > div:nth-child(1) > div:nth-child(1) > div > lib-link-selector > div > div > div > a:nth-child(3)").click()
    except:    
        print('No encuentro el boton de Monthly')
    sleep(2)
    urlactual= driver.current_url
    print(urlactual)
    print('Estoy en la pagina')
    html_table_page = driver.page_source
    
    # for dia in range(1, 11): 
    #     url_modificada = urlactual[:-2] + str(dia)
    #     driver.get(url_modificada)
    #     html_table_page = driver.page_source
    #     sopa = BeautifulSoup(html_table_page)
    #     tabla=sopa.findAll("table",{'aria-labelledby':'Days data'})
    sopa = BeautifulSoup(html_table_page)
    tabla=sopa.findAll("table",{'aria-labelledby':'Days data'})
    print(tabla)
    break








https://www.wunderground.com/history/monthly/LEMD/date/2024-10
Estoy en la pagina
[<table _ngcontent-app-root-c196="" aria-labelledby="Days data"><tr _ngcontent-app-root-c196="" class="ng-star-inserted"><td _ngcontent-app-root-c196="" class="ng-star-inserted"> Oct </td><!-- --></tr><tr _ngcontent-app-root-c196="" class="ng-star-inserted"><td _ngcontent-app-root-c196="" class="ng-star-inserted"> 1 </td><!-- --></tr><tr _ngcontent-app-root-c196="" class="ng-star-inserted"><td _ngcontent-app-root-c196="" class="ng-star-inserted"> 2 </td><!-- --></tr><tr _ngcontent-app-root-c196="" class="ng-star-inserted"><td _ngcontent-app-root-c196="" class="ng-star-inserted"> 3 </td><!-- --></tr><tr _ngcontent-app-root-c196="" class="ng-star-inserted"><td _ngcontent-app-root-c196="" class="ng-star-inserted"> 4 </td><!-- --></tr><tr _ngcontent-app-root-c196="" class="ng-star-inserted"><td _ngcontent-app-root-c196="" class="ng-star-inserted"> 5 </td><!-- --></tr><tr _ngcontent-app-root-c196="" class="ng-

[<table _ngcontent-app-root-c196="" aria-labelledby="Days data"><tr _ngcontent-app-root-c196="" class="ng-star-inserted"><td _ngcontent-app-root-c196="" class="ng-star-inserted"> Oct </td><!-- --></tr><tr _ngcontent-app-root-c196="" class="ng-star-inserted"><td _ngcontent-app-root-c196="" class="ng-star-inserted"> 1 </td><!-- --></tr><tr _ngcontent-app-root-c196="" class="ng-star-inserted"><td _ngcontent-app-root-c196="" class="ng-star-inserted"> 2 </td><!-- --></tr><tr _ngcontent-app-root-c196="" class="ng-star-inserted"><td _ngcontent-app-root-c196="" class="ng-star-inserted"> 3 </td><!-- --></tr><tr _ngcontent-app-root-c196="" class="ng-star-inserted"><td _ngcontent-app-root-c196="" class="ng-star-inserted"> 4 </td><!-- --></tr><tr _ngcontent-app-root-c196="" class="ng-star-inserted"><td _ngcontent-app-root-c196="" class="ng-star-inserted"> 5 </td><!-- --></tr><tr _ngcontent-app-root-c196="" class="ng-star-inserted"><td _ngcontent-app-root-c196="" class="ng-star-inserted"> 6 </td><!

In [166]:
lista_df=[]
for subtabla in tabla:
    df=pd.DataFrame([subtabla.findChildren('tr',recursive=False)[i].text for i in range(len(subtabla.findChildren('tr',recursive=False)))])
    lista_df.append(df)
df_final=pd.concat(lista_df, axis=1)
df_final.columns=['Dia','Temperature','Dew Point','Humidity','Wind Speed','Pressure','Precipitation']
df_final1=df_final.iloc[1:]
df_final1[['Temp Max', 'Temp Avg', 'Temp Min']] = df_final1['Temperature'].str.split(expand=True)
df_final1.drop(columns='Temperature',inplace=True)
df_final1[['DP Max', 'DP Avg', 'DP Min']] = df_final1['Dew Point'].str.split(expand=True)
df_final1.drop(columns='Dew Point',inplace=True)
df_final1[['Hum Max', 'Hum Avg', 'Hum Min']] = df_final1['Humidity'].str.split(expand=True)
df_final1.drop(columns='Humidity',inplace=True)
df_final1[['WS Max', 'WS Avg', 'WS Min']] = df_final1['Wind Speed'].str.split(expand=True)
df_final1.drop(columns='Wind Speed',inplace=True)
df_final1[['Pres Max', 'Pres Avg', 'Pres Min']] = df_final1['Pressure'].str.split(expand=True)
df_final1.drop(columns='Pressure',inplace=True)
columna_a_mover = df_final1['Precipitation']
df_final1.drop(columns='Precipitation',inplace=True) 
df_final1['Precipitation']= columna_a_mover 

C:\Users\alexc\AppData\Local\Temp\ipykernel_9904\676657372.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final1[['Temp Max', 'Temp Avg', 'Temp Min']] = df_final1['Temperature'].str.split(expand=True)
C:\Users\alexc\AppData\Local\Temp\ipykernel_9904\676657372.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final1[['Temp Max', 'Temp Avg', 'Temp Min']] = df_final1['Temperature'].str.split(expand=True)
C:\Users\alexc\AppData\Local\Temp\ipykernel_9904\676657372.py:8: SettingWithCopyWarning: 
A va

In [159]:
# df_final=pd.concat(lista_df, axis=1)
# df_final.columns=['Dia','Temperature','Dew Point','Humidity','Wind Speed','Pressure','Precipitation']
# df_final1=df_final.iloc[1:]

In [167]:
# df_final1[['Temp Max', 'Temp Avg', 'Temp Min']] = df_final1['Temperature'].str.split(expand=True)
# df_final1.drop(columns='Temperature',inplace=True)
# df_final1[['DP Max', 'DP Avg', 'DP Min']] = df_final1['Dew Point'].str.split(expand=True)
# df_final1.drop(columns='Dew Point',inplace=True)
# df_final1[['Hum Max', 'Hum Avg', 'Hum Min']] = df_final1['Humidity'].str.split(expand=True)
# df_final1.drop(columns='Humidity',inplace=True)
# df_final1[['WS Max', 'WS Avg', 'WS Min']] = df_final1['Wind Speed'].str.split(expand=True)
# df_final1.drop(columns='Wind Speed',inplace=True)
# df_final1[['Pres Max', 'Pres Avg', 'Pres Min']] = df_final1['Pressure'].str.split(expand=True)
# df_final1.drop(columns='Pressure',inplace=True)
# columna_a_mover = df_final1['Precipitation']
# df_final1.drop(columns='Precipitation',inplace=True) 
# df_final1['Precipitation']= columna_a_mover 

In [168]:
df_final1

,Dia,Temp Max,Temp Avg,Temp Min,DP Max,DP Avg,DP Min,Hum Max,Hum Avg,Hum Min,WS Max,WS Avg,WS Min,Pres Max,Pres Avg,Pres Min,Precipitation
1,1,84,66.8,50,55,45.2,41,72,48.8,23,10,3.6,0,28.0,27.9,27.9,0.00
2,2,79,71.2,63,63,57.0,52,78,62.0,47,18,8.8,0,27.9,27.8,27.7,0.00
3,3,77,67.6,61,59,49.6,37,88,55.5,24,16,5.6,0,27.9,27.8,27.7,0.00
4,4,77,63.9,50,50,46.1,43,82,53.9,36,8,2.3,0,27.9,27.9,27.8,0.00
5,5,81,65.5,50,59,51.0,45,87,61.8,39,17,5.9,0,27.9,27.9,27.8,0.00
6,6,75,69.7,64,59,56.7,55,73,64.1,50,17,12.7,1,27.8,27.8,27.8,0.00
7,7,75,63.6,57,57,53.6,48,94,71.8,38,24,6.9,0,27.8,27.7,27.6,0.00
8,8,64,60.0,55,57,54.6,52,88,82.0,68,17,11.4,3,27.8,27.8,27.7,0.00
9,9,70,63.0,59,63,57.3,50,94,81.4,63,23,12.2,3,27.8,27.6,27.5,0.00
10,10,68,60.4,52,50,43.9,41,76,55.4,37,20,7.3,0,27.9,27.8,27.7,0.00
